In [ ]:

import random
import networkx as nx

try:
    gph = nx.read_gexf("./microGraph.gexf")
    print("graph file found at " + str())
    print(gph)

except Exception as e: # if any error
    print(e)
    gph = nx.Graph()

values = {}
contributions = set() # all nodes with type contribution
starred = set() # all nodes with type starred repo

for n in gph.nodes(data = True):
    if n[1]['bipartite'] == 0:
        values[n[0]] = 1
    else:
        values[n[0]] = 2

for n in gph.edges(data = True):
    if 'contributions' in n[2]:
        contributions.add(n[0])
        contributions.add(n[1])
    else:
        starred.add(n[0])
        starred.add(n[1])

# users -> 1
# issues -> 2
# contribution -> 3
# starred -> 4

threshold = 3
for node in gph.nodes():
    if values[node] == 1: # if it is user
        current_user = node
        weight = {} # to store weights of issues
        prediction = []
        for nodes in gph.neighbors(current_user):
            if values[nodes] == 2: # issues connected to users
                issues = nodes
                for issue_neigh in gph.neighbors(issues): # neighbours of issues connected to users
                    if values[issue_neigh] == 2:
                        if issue_neigh in weight:
                            weight[issue_neigh] += 1
                        else:
                            weight[issue_neigh] = 1
            else: # users connected to users
                user = nodes
                if user in contributions: # if user is connected via contribution
                    for issue_neigh in gph.neighbors(user): # issues that are neighbours of users connected to cuurent user 
                        if values[issue_neigh] == 2:
                            if issue_neigh in weight:
                                weight[issue_neigh] += 2
                            else:
                                weight[issue_neigh] = 2
                else: # if user is connected via starred repo
                    for issue_neigh in gph.neighbors(user): # issues that are neighbours of users connected to cuurent user
                        if values[issue_neigh] == 2:
                            if issue_neigh in weight:
                                weight[issue_neigh] += 1
                            else:
                                weight[issue_neigh] = 1
        
        Issues = list(weight.keys())
        Weights = list(weight.values())

        print(current_user, end = " --> ")
        # Probability = Element Weight / Sum Of All Weights
        print(random.choices(Issues, Weights, k = 1)) # k is the number of outputs
        
# -------------------------------------------------------------------------------------------------------------------------------------------
# just based on some threshold
        # for issue, value in weight.items():
        #     if value >= threshold:
        #         prediction.append(issue)
        # print(user, end = " --> ")
        # print(prediction)